<a href="https://colab.research.google.com/github/AishwaryaPonni/Madrix_text_analysis/blob/main/emotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Loading the packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
#for stopwords removal
import nltk
from nltk.corpus import stopwords 
#for regularising the input sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
#for train-test split
from sklearn.model_selection import train_test_split
#for model building and testing
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

Reading and analysing the dataset

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/IETE text analysis /Emotion Dataset.csv')

In [ ]:
df.head()

,Unnamed: 0,Sentence,Emotion
0,0,This did the trick : the boys now have a more ...,happy
1,1,"When Anna left Inspector Aziz , she was much h...",happy
2,2,"And though , as Lachlan had planned , they had...",happy
3,3,"Honestly , I 'm really happy for you",happy
4,4,Lesley was totally happy about it .,happy


In [ ]:
df['Emotion'].value_counts()

sad         468
anger       284
fear        279
happy       268
surprise    160
shame        78
disgust      57
Name: Emotion, dtype: int64

In [ ]:
df2 = pd.read_csv('/content/gdrive/MyDrive/IETE text analysis /dataset/full.csv')

In [ ]:
df2['Emotion'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: Emotion, dtype: int64

Removing the stop words

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#removing stop words
def remove_stopwords(review_list):
  stop = stopwords.words('english')
  removed = []
  for review in review_list:
    removed.append(' '.join([word for word in review.split() if word not in stop]))
  return removed

In [ ]:
df2['Sentence'] = remove_stopwords(df2['Sentence'])

In [ ]:
dataset = df2

In [ ]:
dataset['Emotion'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: Emotion, dtype: int64

In [ ]:
joy = dataset[dataset['Emotion'] == 'joy'][:1500]
sadness = dataset[dataset['Emotion'] == 'sadness'][:1500]
anger = dataset[dataset['Emotion'] == 'anger'][:1500]
fear = dataset[dataset['Emotion'] == 'fear'][:1500]
love = dataset[dataset['Emotion'] == 'love'][:1500]
surprise = dataset[dataset['Emotion'] == 'surprise'][:1500]

In [ ]:
d = {'Sentence' : np.array(surprise)[:1500,0], 'Emotion' : np.array(surprise)[:1500,1]}
surprise = pd.DataFrame(d)
joy=joy.append(surprise, ignore_index = True)
joy['Emotion'].value_counts()

love        1500
anger       1500
fear        1500
sadness     1500
joy         1500
surprise    1438
Name: Emotion, dtype: int64

In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/IETE text analysis\ 

In [ ]:
dataset = joy
dataset.to_csv('/content/gdrive/MyDrive/IETE text analysis /balanced_emotion_dataset.csv')

In [ ]:
dataset['Emotion'].value_counts()

love        1500
anger       1500
fear        1500
sadness     1500
joy         1500
surprise    1438
Name: Emotion, dtype: int64

Data Sequencing and splitting

In [ ]:
max_words = 5000
data = dataset.Sentence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
sentences = pad_sequences(sequences)
print(sentences.shape)

(8219, 33)


Label Encoding

In [ ]:
labels = np.array(dataset['Emotion'])
y = []
for i in range(len(labels)):
    if labels[i] == 'joy':
        y.append(0)
    if labels[i] == 'sadness':
        y.append(1)
    if labels[i] == 'anger':
        y.append(2)
    if labels[i] == 'fear':
        y.append(3)
    if labels[i] == 'love':
        y.append(4)
    if labels[i] == 'surprise':
        y.append(5)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 6, dtype="float32")

In [ ]:
labels.shape

(8219, 6)

Splitting into train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences,
                                                    labels,
                                                    test_size=0.15,
                                                    random_state=42)

In [ ]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1,   97,  527,  516,  756,   16,   38,  982, 1019,  165,  171],
      dtype=int32)

In [ ]:
print (len(X_train),len(X_test),len(y_train),len(y_test))

6986 1233 6986 1233


Model building and testing
(With single LSTM layer)

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) #The embedding layer
model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(6,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

# checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='max', save_freq=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test)) #,callbacks=[checkpoint1])

Epoch 1/20
219/219 [==============================] - 3s 8ms/step - loss: 1.7728 - accuracy: 0.2059 - val_loss: 1.7515 - val_accuracy: 0.2563
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 1.7208 - accuracy: 0.3466 - val_loss: 1.6436 - val_accuracy: 0.3114
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 1.5298 - accuracy: 0.4534 - val_loss: 1.4727 - val_accuracy: 0.3496
Epoch 4/20
219/219 [==============================] - 1s 5ms/step - loss: 1.2246 - accuracy: 0.6039 - val_loss: 1.0901 - val_accuracy: 0.6821
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.9452 - accuracy: 0.7237 - val_loss: 0.8404 - val_accuracy: 0.7470
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.7327 - accuracy: 0.7891 - val_loss: 0.6667 - val_accuracy: 0.8224
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.5662 - accuracy: 0.8427 - val_loss: 0.5978 - val_accuracy: 0.8102
Epoch 

Function for prediction

In [ ]:

def prediction(text):
  emotion = ['joy','sadness','anger','fear','love','surprise']
  # l = list(text)
  sequences = tokenizer.texts_to_sequences([text])
  test = pad_sequences(sequences)
  return emotion[np.around(model1.predict(test), decimals=0).argmax(axis=1)[0]]

In [ ]:
prediction('i am feeling very depressed')

'sadness'

In [ ]:
prediction('feeling very happy today')

'joy'

In [ ]:
prediction('i am so mad at you')

'anger'

In [ ]:
prediction('i am so scared of you')

'fear'

In [ ]:
prediction('i was stunned by his performance tonight')

'surprise'

In [ ]:
prediction('i feel blessed to have a friend like you')

'love'

In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/IETE text analysis /emotion_analyser_english')
emotion[np.around(new_model.predict(test), decimals=0).argmax(axis=1)[0]]